In [1]:
import sys,os,re
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd
import seaborn           as sb



from itertools import product
from copy      import deepcopy
from time      import time

from scipy.interpolate import interp1d
from scipy.integrate   import trapz

import warnings
warnings.filterwarnings('ignore')

import matplotlib
from matplotlib import rc
from matplotlib.pyplot import cm
from matplotlib.colors import LogNorm

rc('text', usetex=True)
rc('font', family='serif')
matplotlib.rcParams.update({'font.size': 18})


sidelegend = {'bbox_to_anchor': (1.04,0.5), 
              'loc': "center left",
              'frameon': False}


In [2]:
fiducial = {'ombh2': 0.022445,
            'omch2': 0.1205579307,
            'ns': 0.96,
            'As': 2.12605e-09,
            'tau': 0.05,
            'H0': 67.36,
            'w': -1.,
            'wa': 0.,
            'mnu': 0.06}
epsilon=0

In [3]:
import camb

z_camb  = np.linspace(0.001,4.,10000)
pars    = camb.set_params(**fiducial)
results = camb.get_results(pars)
dA      = results.angular_diameter_distance2(0,z_camb)
Hz      = results.hubble_parameter(z_camb)
dL_camb = results.luminosity_distance(z_camb)
dL      = (1+z_camb)**(2+epsilon)*dA

AttributeError: can't set attribute

In [ ]:


BAO_mock = {'DH': interp1d(z_camb,  2.99 * 10**5/ Hz),
            'DV': interp1d(z_camb, ((1+z_camb)**2 * dA**2 *z_camb * 2.99 * 10**5/ Hz )**(1/3))}

In [ ]:
bin_edges = np.linspace(0.65, 1.85, 14)
z_bins    = (bin_edges[:-1] + bin_edges[1:]) / 2
DH        = BAO_mock['DH'](z_bins)
DV        = BAO_mock['DV'](z_bins)


In [ ]:
y1 = 2.99*10**5 / (DH)
y2 = (DV**3/(z_bins*(1+z_bins)**2 * DH))**0.5
sigma= np.abs(np.random.normal(loc=0, scale=100, size=len(z_bins) ))

plt.figure(figsize=(8, 6))
plt.plot(z_camb, dA, label="Theoretical dA", color='red', linewidth=2)
plt.errorbar(z_bins, y2, yerr=sigma, fmt='o',label='dA mock')
plt.xlim(0.3, 2.0)
plt.ylim(1300, 2100)
plt.xlabel("Redshift (z)")
plt.ylabel("DA")
plt.title("DA vs Redshift")
plt.legend()
plt.show()

In [ ]:
sigma = np.abs(np.random.normal(loc=0, scale=10, size=len(z_bins) ))
plt.figure(figsize=(8, 6))
plt.plot(z_camb, Hz, label="Theoretical Hz", color='red', linewidth=2)
plt.errorbar(z_bins, y1, yerr=sigma, fmt='o',label='Hz mock')
plt.xlim(0.3, 2.0)
plt.ylim(50, 250)
plt.xlabel("Redshift (z)")
plt.ylabel("Hz")
plt.title("Hz vs Redshift")
plt.legend()
plt.show()

In [ ]:
#SN
N_SN = 8800

def dN_dz(z):
    return 1.53e-4 * ((1 + z) / 1.5) ** 2.14 * (fiducial['H0']/70)**3


z         = np.linspace(0.1,1.0, 10000)
p_z       = dN_dz(z)
p_z      /= np.sum(p_z) 
z_sampled = np.random.choice(z, size=N_SN, p=p_z, replace=False)

z_SN      = np.sort(z_sampled)



mock_mu   = interp1d(z_camb, 5 * np.log10(dL) + 25)

mu        = mock_mu(z_SN)



In [ ]:
mu_theoretical = 5 * np.log10(dL_camb) + 25


In [ ]:
sigma = np.abs(np.random.normal(loc=0, scale= 0.2, size=N_SN) )
z_plot= np.linspace(0.1,1.0,10000)


plt.figure(figsize=(8, 6))
plt.errorbar(z_SN, mu, fmt='o', yerr=sigma, label="Distance Modulus vs Redshift", 
             capsize=2, markersize=5, alpha=0.8,  zorder=1)

plt.plot(z_camb, mu_theoretical, label="Theoretical Distance Modulus", color='red', linewidth=3, zorder=2)

plt.xlim(0.1,1.0)
plt.ylim(35,45)
plt.xlabel("Redshift (z)")
plt.ylabel("Distance Modulus (mu)")
plt.title("Distance Modulus vs Redshift")
plt.legend()
plt.show()

In [ ]:
#SAVING DATA
sigma_SN = [1] * N_SN
df_SN        = pd.DataFrame({'z': z_SN,
                             'mu': mu,
                             'err': sigma_SN})

df_SN.to_csv('SN_mock.txt', sep='\t', index=False)

sigma_DH= [10] * len(z_bins)
sigma_DV= [10] * len(z_bins)

df_BAO        = pd.DataFrame({'z' : z_bins,
                             'DH': DH,
                             'err_DH': sigma_DH,
                             'DV': DV,
                             'err_DV': sigma_DV})
#print(df)

df_BAO.to_csv('BAO_mock.txt', sep='\t', index=False)